In [1]:
# import
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

C:\ProgramData\Anaconda3\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.26.0
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [43]:
train=pd.read_csv("C:/Users/82102/Desktop/의료/128.치매 고위험군 라이프로그/128.치매 고위험군 라이프로그/01.데이터/train_data.csv")
val=pd.read_csv("C:/Users/82102/Desktop/의료/128.치매 고위험군 라이프로그/128.치매 고위험군 라이프로그/01.데이터/val_data.csv")

In [44]:
train.drop(columns=['Unnamed: 0'], inplace=True)
val.drop(columns=['Unnamed: 0'], inplace=True)

In [45]:
CN_data=train[train['DIAG_NM']=='CN']
MCI_data=train[train['DIAG_NM']=='MCI']
Dem_data=train[train['DIAG_NM']=='Dem']
CN_data=CN_data.reset_index(drop=True)
MCI_data=MCI_data.reset_index(drop=True)
Dem_data=Dem_data.reset_index(drop=True)

# 치매 환자 확인

In [46]:
Dem_data

,EMAIL,activity_average_met,activity_cal_active,activity_cal_total,activity_daily_movement,activity_day_end,activity_day_start,activity_high,activity_inactive,activity_inactivity_alerts,...,Q14_1,Q14_2,Q15,Q16_1,Q16_2,Q16_3,Q17,Q18,Q19,TOTAL
0,nia+067@rowan.kr,1.43750,397,2472,7103,2020-10-19T03:59:59+09:00,2020-10-18T04:00:00+09:00,0,636,2,...,2,2,2,1,1,2,1,2,2,16
1,nia+067@rowan.kr,1.37500,366,2398,6881,2020-10-20T03:59:59+09:00,2020-10-19T04:00:00+09:00,0,586,2,...,2,2,2,1,1,2,1,2,2,16
2,nia+067@rowan.kr,1.34375,330,2354,6389,2020-10-21T03:59:59+09:00,2020-10-20T04:00:00+09:00,0,539,1,...,2,2,2,1,1,2,1,2,2,16
3,nia+067@rowan.kr,1.21875,92,2122,1159,2020-10-22T03:59:59+09:00,2020-10-21T04:00:00+09:00,0,647,0,...,2,2,2,1,1,2,1,2,2,16
4,nia+067@rowan.kr,1.25000,206,2188,3755,2020-10-23T03:59:59+09:00,2020-10-22T04:00:00+09:00,0,506,1,...,2,2,2,1,1,2,1,2,2,16
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
566,nia+196@rowan.kr,1.00000,14,1845,194,2021-01-02T03:59:59+09:00,2021-01-01T04:00:00+09:00,0,104,0,...,2,2,2,1,2,1,2,1,2,16
567,nia+196@rowan.kr,1.15625,71,2032,1090,2021-01-03T03:59:59+09:00,2021-01-02T04:00:00+09:00,0,581,1,...,2,2,2,1,2,1,2,1,2,16
568,nia+196@rowan.kr,1.21875,207,2162,3726,2021-01-04T03:59:59+09:00,2021-01-03T04:00:00+09:00,2,457,0,...,2,2,2,1,2,1,2,1,2,16
569,nia+196@rowan.kr,1.15625,122,2071,1974,2021-01-05T03:59:59+09:00,2021-01-04T04:00:00+09:00,0,507,0,...,2,2,2,1,2,1,2,1,2,16


# hypnogram 열 빼고 다 제거

In [47]:
Dem_data=Dem_data[['EMAIL','CONVERT(sleep_hypnogram_5min USING utf8)']]

In [48]:
Dem_hypnogram = {}
for email in Dem_data['EMAIL'].unique():
    Dem_hypnogram[email] = Dem_data[Dem_data['EMAIL'] == email]

In [49]:
Dem_hypnogram

{'nia+067@rowan.kr':                EMAIL           CONVERT(sleep_hypnogram_5min USING utf8)
 0   nia+067@rowan.kr  4/2/4/4/2/2/2/2/2/2/2/2/2/2/2/2/4/2/2/1/2/3/3/...
 1   nia+067@rowan.kr  4/2/2/3/2/2/2/2/2/1/2/2/2/2/2/2/2/2/2/2/2/2/2/...
 2   nia+067@rowan.kr  4/4/4/2/2/4/4/2/2/1/1/1/2/2/4/4/2/2/4/4/4/4/4/...
 3   nia+067@rowan.kr  4/4/4/2/4/4/2/4/4/2/2/2/2/1/1/2/2/3/4/4/4/2/2/...
 4   nia+067@rowan.kr  4/2/1/2/4/4/2/2/2/1/1/1/1/1/4/4/4/2/2/2/4/4/2/...
 ..               ...                                                ...
 70  nia+067@rowan.kr  4/2/2/2/2/1/1/1/1/2/2/2/2/2/1/2/4/4/2/1/1/1/2/...
 71  nia+067@rowan.kr  4/4/2/2/2/2/2/2/1/1/4/2/2/1/1/2/2/3/3/2/4/4/4/...
 72  nia+067@rowan.kr  4/2/2/2/2/2/2/2/2/2/2/2/1/1/2/2/2/2/2/2/2/2/2/...
 73  nia+067@rowan.kr  4/2/2/2/2/2/2/4/2/2/2/2/2/2/1/1/4/4/4/2/2/2/3/...
 74  nia+067@rowan.kr  2/4/4/4/2/2/2/1/1/1/2/2/2/2/2/2/2/2/2/2/2/2/2/...
 
 [75 rows x 2 columns],
 'nia+341@rowan.kr':                 EMAIL           CONVERT(sleep_hypnogram_5

In [52]:
Dem_concatenated_hypnogram = {}

# 각 이메일 주소별로 데이터를 이어붙임
for email, group in Dem_data.groupby('EMAIL'):
    Dem_concatenated_hypnogram[email] = ''.join(group['CONVERT(sleep_hypnogram_5min USING utf8)'].str.replace('/', ''))

In [53]:
Dem_concatenated_hypnogram

{'nia+017@rowan.kr': '232332221121111113333332222221221111111122433222222222222222222222242233222221111133323333322244442332344444222222222112244444332444222233442222111122222222233332222112222211111123324223324423324442422111322332221222222211111122223333222221111222112222222442422222222333333324444444132244444444222222222222111333322222222222222222332222222111113324444444422222424322222211112224432442222211111144422222222224224424444322222223344442122222222444444442222442222222233322133322111132332222442222211222223232332224222222222211333444444422222122233322211111111422232232441333233332244424442222222222223322222222222333232222444422233422111111243234422222211111222333222221322222222114444422211211123333333222211112223332444224222221111222324232222211111111222222444422222322333333322233224442222222222332223344441122112213322211111111333342222222223333222223222222332222211122112224442222222442211112333322222232222223332221111113222222242224442222221112222222222232222221122222222233

In [54]:
# 이메일, 길이, 1과 3이 연속해서 나오는 횟수, 3과 1이 연속해서 나오는 횟수를 저장할 리스트 초기화
emails = []
lengths = []
count_13 = []
count_31 = []

# 각 이메일 주소별로 데이터를 처리하여 리스트에 저장
for email, hypnogram in Dem_concatenated_hypnogram.items():
    emails.append(email)
    lengths.append(len(hypnogram))

    # 연속된 1과 3, 3과 1의 횟수 초기화
    count_13.append(0)
    count_31.append(0)

    # hypnogram 문자열을 순회하면서 연속된 숫자 쌍을 찾음
    for i in range(len(hypnogram) - 1):
        if hypnogram[i:i+2] == '13':
            count_13[-1] += 1
        elif hypnogram[i:i+2] == '31':
            count_31[-1] += 1

# 데이터프레임 생성
Dem_hypnogram_count = pd.DataFrame({
    'EMAIL': emails,
    'Length': lengths,
    'Count_13': count_13,
    'Count_31': count_31
})

# 1과 3이 연속해서 나오는 횟수와 3과 1이 연속해서 나오는 횟수를 더한 값을 새로운 열로 추가
Dem_hypnogram_count['Total_Count'] = Dem_hypnogram_count['Count_13'] + Dem_hypnogram_count['Count_31']

# 길이에 따른 1-3, 3-1 연속적으로 나타나는 횟수 비율
Dem_hypnogram_count['Rating'] = (Dem_hypnogram_count['Count_13'] + Dem_hypnogram_count['Count_31'])/Dem_hypnogram_count['Length']

# 결과 확인
Dem_hypnogram_count

,EMAIL,Length,Count_13,Count_31,Total_Count,Rating
0,nia+017@rowan.kr,5032,40,3,43,0.008545
1,nia+067@rowan.kr,9330,24,30,54,0.005788
2,nia+139@rowan.kr,6037,10,4,14,0.002319
3,nia+196@rowan.kr,10776,33,26,59,0.005475
4,nia+219@rowan.kr,3964,23,18,41,0.010343
5,nia+230@rowan.kr,10745,88,67,155,0.014425
6,nia+306@rowan.kr,7814,41,23,64,0.008190
7,nia+338@rowan.kr,4990,19,15,34,0.006814
8,nia+341@rowan.kr,6106,71,32,103,0.016869


In [55]:
print("치매 환자들의 평균 Rating : "+ str(Dem_hypnogram_count['Rating'].mean()))

치매 환자들의 평균 Rating : 0.008752040475790765


# 정상인

In [56]:
CN_data=CN_data[['EMAIL','CONVERT(sleep_hypnogram_5min USING utf8)']]

In [57]:
CN_hypnogram = {}
for email in CN_data['EMAIL'].unique():
    CN_hypnogram[email] = CN_data[CN_data['EMAIL'] == email]

In [58]:
CN_concatenated_hypnogram = {}

# 각 이메일 주소별로 데이터를 이어붙임
for email, group in CN_data.groupby('EMAIL'):
    CN_concatenated_hypnogram[email] = ''.join(group['CONVERT(sleep_hypnogram_5min USING utf8)'].str.replace('/', ''))

In [59]:
# 이메일, 길이, 1과 3이 연속해서 나오는 횟수, 3과 1이 연속해서 나오는 횟수를 저장할 리스트 초기화
emails = []
lengths = []
count_13 = []
count_31 = []

# 각 이메일 주소별로 데이터를 처리하여 리스트에 저장
for email, hypnogram in CN_concatenated_hypnogram.items():
    emails.append(email)
    lengths.append(len(hypnogram))

    # 연속된 1과 3, 3과 1의 횟수 초기화
    count_13.append(0)
    count_31.append(0)

    # hypnogram 문자열을 순회하면서 연속된 숫자 쌍을 찾음
    for i in range(len(hypnogram) - 1):
        if hypnogram[i:i+2] == '13':
            count_13[-1] += 1
        elif hypnogram[i:i+2] == '31':
            count_31[-1] += 1

# 데이터프레임 생성
CN_hypnogram_count = pd.DataFrame({
    'EMAIL': emails,
    'Length': lengths,
    'Count_13': count_13,
    'Count_31': count_31
})

# 1과 3이 연속해서 나오는 횟수와 3과 1이 연속해서 나오는 횟수를 더한 값을 새로운 열로 추가
CN_hypnogram_count['Total_Count'] = CN_hypnogram_count['Count_13'] + CN_hypnogram_count['Count_31']

# 길이에 따른 1-3, 3-1 연속적으로 나타나는 횟수 비율
CN_hypnogram_count['Rating'] = (CN_hypnogram_count['Count_13'] + CN_hypnogram_count['Count_31'])/CN_hypnogram_count['Length']

# 결과 확인
CN_hypnogram_count

,EMAIL,Length,Count_13,Count_31,Total_Count,Rating
0,nia+005@rowan.kr,11389,109,39,148,0.012995
1,nia+015@rowan.kr,3331,17,12,29,0.008706
2,nia+021@rowan.kr,6743,34,30,64,0.009491
3,nia+031@rowan.kr,9138,109,60,169,0.018494
4,nia+032@rowan.kr,7396,33,8,41,0.005544
...,...,...,...,...,...,...
80,nia+445@rowan.kr,2002,23,7,30,0.014985
81,nia+451@rowan.kr,3467,39,17,56,0.016152
82,nia+456@rowan.kr,6136,41,31,72,0.011734
83,nia+458@rowan.kr,6198,38,46,84,0.013553


In [60]:
print("일반인의 평균 Rating : "+ str(CN_hypnogram_count['Rating'].mean()))

일반인의 평균 Rating : 0.013613963195989431


# 경증 환자

In [61]:
MCI_data=MCI_data[['EMAIL','CONVERT(sleep_hypnogram_5min USING utf8)']]

In [62]:
MCI_hypnogram = {}
for email in MCI_data['EMAIL'].unique():
    MCI_hypnogram[email] = MCI_data[CN_data['EMAIL'] == email]

C:\Users\82102\AppData\Local\Temp/ipykernel_20708/419365315.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  MCI_hypnogram[email] = MCI_data[CN_data['EMAIL'] == email]


In [63]:
MCI_concatenated_hypnogram = {}

# 각 이메일 주소별로 데이터를 이어붙임
for email, group in MCI_data.groupby('EMAIL'):
    MCI_concatenated_hypnogram[email] = ''.join(group['CONVERT(sleep_hypnogram_5min USING utf8)'].str.replace('/', ''))

In [64]:
# 이메일, 길이, 1과 3이 연속해서 나오는 횟수, 3과 1이 연속해서 나오는 횟수를 저장할 리스트 초기화
emails = []
lengths = []
count_13 = []
count_31 = []

# 각 이메일 주소별로 데이터를 처리하여 리스트에 저장
for email, hypnogram in MCI_concatenated_hypnogram.items():
    emails.append(email)
    lengths.append(len(hypnogram))

    # 연속된 1과 3, 3과 1의 횟수 초기화
    count_13.append(0)
    count_31.append(0)

    # hypnogram 문자열을 순회하면서 연속된 숫자 쌍을 찾음
    for i in range(len(hypnogram) - 1):
        if hypnogram[i:i+2] == '13':
            count_13[-1] += 1
        elif hypnogram[i:i+2] == '31':
            count_31[-1] += 1

# 데이터프레임 생성
MCI_hypnogram_count = pd.DataFrame({
    'EMAIL': emails,
    'Length': lengths,
    'Count_13': count_13,
    'Count_31': count_31
})

# 1과 3이 연속해서 나오는 횟수와 3과 1이 연속해서 나오는 횟수를 더한 값을 새로운 열로 추가
MCI_hypnogram_count['Total_Count'] = MCI_hypnogram_count['Count_13'] + MCI_hypnogram_count['Count_31']

# 길이에 따른 1-3, 3-1 연속적으로 나타나는 횟수 비율
MCI_hypnogram_count['Rating'] = (MCI_hypnogram_count['Count_13'] + MCI_hypnogram_count['Count_31'])/MCI_hypnogram_count['Length']

# 결과 확인
MCI_hypnogram_count

,EMAIL,Length,Count_13,Count_31,Total_Count,Rating
0,nia+003@rowan.kr,6477,78,62,140,0.021615
1,nia+010@rowan.kr,5337,30,11,41,0.007682
2,nia+014@rowan.kr,6495,33,14,47,0.007236
3,nia+016@rowan.kr,5706,22,6,28,0.004907
4,nia+037@rowan.kr,5732,27,15,42,0.007327
5,nia+043@rowan.kr,9060,73,51,124,0.013687
6,nia+044@rowan.kr,6619,45,18,63,0.009518
7,nia+046@rowan.kr,6150,66,59,125,0.020325
8,nia+059@rowan.kr,10797,77,56,133,0.012318
9,nia+061@rowan.kr,7312,30,14,44,0.006018


In [65]:
print("경증의 평균 Rating : "+ str(MCI_hypnogram_count['Rating'].mean()))

경증의 평균 Rating : 0.011919197575439584


# hypnogram 결과

In [67]:
print("일반인의 평균 Rating : "+ str(CN_hypnogram_count['Rating'].mean()))
print("경증의 평균 Rating : "+ str(MCI_hypnogram_count['Rating'].mean()))
print("치매 환자의 평균 Rating : "+ str(Dem_hypnogram_count['Rating'].mean()))

일반인의 평균 Rating : 0.013613963195989431
경증의 평균 Rating : 0.011919197575439584
치매 환자의 평균 Rating : 0.008752040475790765
